In [3]:
import pandas as pd
import re

In [4]:
def clean_numeric(text):
    if isinstance(text, str):
        return re.sub(r'[^0-9.]', '', text)
    return text


In [5]:
def clean_text(text):
    if isinstance(text, str):
        # Mantener letras, números, espacios y tildes
        # Eliminar saltos de línea
        text = re.sub(r'\n', '', text)
        return re.sub(r'[^A-Za-z0-9\sÁÉÍÓÚáéíóúñÑ]', '', text).strip()
    return text


In [6]:
def clean_date(text):
    if isinstance(text, str):
        # Eliminar caracteres que no sean números o guiones
        text = re.sub(r'[^0-9-]', '', text) 
    return text


In [7]:
def clean_email(text):
    if isinstance(text, str):
        # Reemplazar caracteres especiales en medio con '@'
        text = re.sub(r'[^A-Za-z0-9@._-]', '@', text)
        
        # Eliminar caracteres especiales de los extremos
        text = re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', text)
        
        # Asegurarse de que solo haya una '@'
        parts = text.split('@')
        if len(parts) > 2:
            text = parts[0] + '@' + ''.join(parts[1:])
        elif len(parts) == 2:
            text = parts[0] + '@' + parts[1]

        # Eliminar caracteres especiales después del '@' excepto para '.' y '-'
        if '@' in text:
            local, domain = text.split('@', 1)
            domain = re.sub(r'[^A-Za-z0-9.-]', '', domain)
            text = f'{local}@{domain}'

        return text
    return text

In [8]:
def clean_phone(numero):
    # Eliminar caracteres especiales excepto números
    numero = re.sub(r'\D', '', numero)
    
    # Quitar los '0' a la izquierda si los hay
    numero = numero.lstrip('0')
    
    # Agregar '1' al principio si hay exactamente 10 números
    if len(numero) == 10:
        numero = '1' + numero
        
    # Agregar '1' al principio si no lo tiene
    if not numero.startswith('1'):
        numero = '1' + numero
    
    # Formatear el número y extraer la extensión si existe
    if len(numero) > 11:  # Si hay más de 11 dígitos, hay una extensión
        formateado = f'+{numero[0]} ({numero[1:4]}) {numero[4:7]}-{numero[7:11]}'
        extension = numero[11:]
    else:
        formateado = f'+{numero[0]} ({numero[1:4]}) {numero[4:7]}-{numero[7:11]}'
        extension = ''
    
    return formateado, extension

In [9]:
def clean_text_advanced(text):
    if isinstance(text, str):
        # Reemplazar caracteres especiales por espacio, excepto letras, números y espacios
        text = re.sub(r'[^A-Za-z0-9ÁÉÍÓÚáéíóúñÑ\s]', ' ', text)
        
        # Reemplazar múltiples espacios por un solo espacio
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text.strip()  # Eliminar espacios en blanco al principio y al final
    return text

In [10]:
df=pd.read_csv(r'./files/datos_ventas.csv',delimiter=",",encoding="ISO-8859-1")

In [11]:
df['ID_Venta']=df['ID_Venta'].apply(clean_numeric)
df['Producto'] = df['Producto'].apply(clean_text)
df['Ciudad'] = df['Ciudad'].apply(clean_text_advanced)
df['Categoria'] = df['Categoria'].apply(clean_text)
df['Precio_Unitario'] = df['Precio_Unitario'].apply(clean_numeric)
df['Cantidad'] = df['Cantidad'].apply(clean_numeric)
df['Fecha_Venta'] = df['Fecha_Venta'].apply(clean_date)
df['Cliente'] = df['Cliente'].apply(clean_text_advanced)
df['Email'] = df['Email'].apply(clean_email)
df[['Telefono', 'Extension']] = df['Telefono'].apply(lambda x: pd.Series(clean_phone(x)))
df['Direccion'] = df['Direccion'].apply(clean_text_advanced)
df['Metodo_Pago'] = df['Metodo_Pago'].apply(clean_text)
df['Estado'] = df['Estado'].apply(clean_text_advanced)
df['Comentario'] = df['Comentario'].apply(clean_text_advanced)
df['Descuento'] = df['Descuento'].apply(clean_numeric)

In [12]:
columns = list(df.columns)
telefono_idx = columns.index('Telefono')
columns.insert(telefono_idx + 1, columns.pop(columns.index('Extension')))
df = df[columns]

In [13]:
output_path = './files/archivo_limpio.xlsx'
df.to_excel(output_path, index=False)